<a href="https://colab.research.google.com/github/bdemchak/cytoscape-jupyter/blob/main/echo/echo_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This test is designed to hammer Cytoscape by sending a long sequence of short operations. On its own, it's an effective test of Jupyter-Bridge -- it verifies that a Colab Notebook can keep a reliable channel to a private Cytoscape.

But when this test is executed in multiple Colabs (each connected to its own Cytoscape in its own VM), it verifies that Jupyter-Bridge can process a high volume of requests while keeping good track of requests and replies.


---
# Setup data files, py4cytoscape and Cytoscape connection
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

## Setup: Fetch latest py4cytoscape




**Note that you can fetch from the latest Github unreleased version by setting _PY4CYTOSCAPE to 'git+https://github.com/cytoscape/py4cytoscape' immediately before the exec() call. To fetch a particular branch, add '@' to the end (e.g., 'git+https://github.com/cytoscape/py4cytoscape@0.0.12').**

To load the default (PyPI) py4cytoscape version, do not set _PY4CYTOSCAPE at all.

In [2]:
_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.6.0' # optional
import requests

exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)

IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cytoscape/py4cytoscape (to revision 1.6.0) to /tmp/pip-req-build-0n1of_4_


  Running command git clone -q https://github.com/cytoscape/py4cytoscape /tmp/pip-req-build-0n1of_4_
  Running command git checkout -b 1.6.0 --track origin/1.6.0
  Switched to a new branch '1.6.0'
  Branch '1.6.0' set up to track remote branch '1.6.0' from 'origin'.


  Created wheel for py4cytoscape: filename=py4cytoscape-1.6.0-py3-none-any.whl size=170883 sha256=9b15eba47f9dc447c9a4f7c6bd915b9b66a491c86d5cae4e167bea54c91b74d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-n5jgn4jm/wheels/41/f8/55/7d9713a08400a67188976c9214b7e48e0da9cd32b53ad30109
  Created wheel for python-igraph: filename=python_igraph-0.9.11-py3-none-any.whl size=9074 sha256=208a2bf6333e3ce2d61ed9d3039b6ab72490a8978f6ee3478acfa791a6c9906c
  Stored in directory: /root/.cache/pip/wheels/7d/e4/0c/0c83f70bd0b99ce8aea47a21f8e52502169e7dd17808d12f30
Successfully built py4cytoscape python-igraph
NumExpr defaulting to 2 threads.
Loading Javascript client ... d3ef5dc4-a593-4bf3-b639-6c1e951191d8 on https://jupyter-bridge.cytoscape.org
ADVICE: WHEN RUNNING UNDER COLAB, DO NOT RE-RUN THIS CELL WITHOUT MANUALLY EXECUTING Runtime | Factory Reset Runtime FROM THE COLAB MENU FIRST.


<IPython.core.display.Javascript object>

## Setup: Sanity test to verify Cytoscape connection

By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.

In [18]:
p4c.cytoscape_version_info()


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.9.1',
 'automationAPIVersion': '1.6.0',
 'py4cytoscapeVersion': '1.6.0',
 'jupyterBridgeVersion': '0.0.2'}

## Setup: Initialize a basic static message
The basic message is a random digit sequence that is unlikely to be used by any other Colab Notebook instance.

In [19]:
import uuid as uu
my_uuid = str(p4c.get_browser_client_channel())

print(f'Fixed message: {my_uuid}')


Fixed message: d3ef5dc4-a593-4bf3-b639-6c1e951191d8


## Test
Call Cytoscape's echo command with a unique message and get back the unique message.

The message consists of a random uuid (previously generated) combined with a monotonically increasing value.

Thus messages sent from this Colab instance are guaranteed to be unique in the universe, and each message sent from this Colab instance is guaranteed to be different from any previous messages.


In [20]:
import py4cytoscape as p4c

for i in range(0, 1000):
  message = f'{i}: {my_uuid}'
  ret = p4c.command_echo(message)
  if message == ret[0]:
    print(f'Successful: {message}')
  else:
    print(f'Failure: got {ret}, expected {message}')


Successful: 0: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 1: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 2: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 3: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 4: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 5: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 6: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 7: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 8: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 9: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 10: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 11: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 12: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 13: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 14: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 15: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 16: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 17: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Successful: 18: d3ef5dc4-a593-4bf3-b639-6c1e951191d8
Suc

In [21]:
res_mitab = p4c.sandbox_url_to("https://www.dropbox.com/s/8wc8o897tsxewt1/BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab?dl=0", "BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab")
res_soft = p4c.sandbox_url_to("https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0", "GDS112_full.soft")

def load_network():
  p4c.close_session(False)
  try:
    p4c.import_network_from_file("BIOGRID-ORGANISM-Saccharomyces_cerevisiea-3.2.105.mitab")
  except:
    pass
  p4c.load_table_data_from_file("GDS112_full.soft", start_load_row=83, data_key_column_index=10, delimiters="\t")

In [22]:
for i in range(0, 10):
  load_network()
  print(f'Load network {i}: {my_uuid} successful')

In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


Load network 0: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


Load network 1: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


...

Load network 7: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


Load network 8: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


Load network 9: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful


In [26]:
load_network()
all_columns = p4c.get_table_columns(table='node', columns=['shared name', 'name', 'psi-mi', 'ID_REF', 'IDENTIFIER', 'GSM1029', 'GSM1030', 'GSM1032', 'GSM1033', 'GSM1034', 'Gene title', 'Gene symbol', 'Platform_ORF', 'Chromosome annotation', 'GO:Function', 'GO:Process', 'GO:Component', 'GO:Function ID', 'GO:Process ID', 'GO:Component ID'])


In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/home/barry/CytoscapeConfiguration/3/framework-cytoscape.log'}


,shared name,name,psi-mi,ID_REF,IDENTIFIER,GSM1029,GSM1030,GSM1032,GSM1033,GSM1034,Gene title,Gene symbol,Platform_ORF,Chromosome annotation,GO:Function,GO:Process,GO:Component,GO:Function ID,GO:Process ID,GO:Component ID
18776351,855232,855232,"""MI:0463""(biogrid)",6219.0,RPL36A,0.873,0.388,-1.871,-0.909,0.006,ribosomal 60S subunit protein L36A,RPL36A,YMR194W,"Chromosome XIII, NC_001145.3 (651145..651910)",RNA binding///poly(A) RNA binding///structural...,cytoplasmic translation///translation,cytoplasm///cytosolic large ribosomal subunit/...,GO:0003723///GO:0044822///GO:0003735///GO:0003735,GO:0002181///GO:0006412,GO:0005737///GO:0022625///GO:0005622///GO:0030...
18841883,853387,853387,"""MI:0463""(biogrid)",6551.0,YHC3,0.116,-0.392,0.019,0.662,0.130,amino acid transporter YHC3,YHC3,YJL059W,"Chromosome X, NC_001142.9 (324964..326190)",molecular_function,amino acid transport///arginine transport///ly...,fungal-type vacuole///integral component of me...,GO:0003674,GO:0006865///GO:0015809///GO:0015819///GO:0051...,GO:0000324///GO:0016021///GO:0016021///GO:0031...
19054886,852070,852070,"""MI:0463""(biogrid)",1271.0,PFA5,0.411,-0.358,-0.288,-0.007,0.447,palmitoyltransferase PFA5,PFA5,YDR459C,"Chromosome IV, NC_001136.10 (1382319..1383443,...",palmitoyltransferase activity///protein-cystei...,peptidyl-L-cysteine S-palmitoylation///protein...,Golgi apparatus///cell periphery///endoplasmic...,GO:0016409///GO:0019706///GO:0019706///GO:0016...,GO:0018230///GO:0018345///GO:0006612,GO:0005794///GO:0071944///GO:0005783///GO:0016...
18759959,853929,853929,"""MI:0463""(biogrid)",256.0,RHO4,0.082,-0.013,0.425,0.374,0.307,Rho family GTPase RHO4,RHO4,YKR055W,"Chromosome XI, NC_001143.9 (548216..549091)",GTP binding///GTPase activity///nucleotide bin...,maintenance of cell polarity///positive regula...,cell periphery///cellular bud neck///endoplasm...,GO:0005525///GO:0003924///GO:0000166,GO:0030011///GO:0090338///GO:0090337///GO:0007264,GO:0071944///GO:0005935///GO:0005783///GO:0000...
18776336,852489,852489,"""MI:0463""(biogrid)",2357.0,RPL21A,0.879,0.371,-0.971,-0.743,0.083,ribosomal 60S subunit protein L21A,RPL21A,YBR191W,"Chromosome II, NC_001134.8 (606270..607140)",poly(A) RNA binding///structural constituent o...,cytoplasmic translation///translation///transl...,cytoplasm///cytosolic large ribosomal subunit/...,GO:0044822///GO:0003735///GO:0003735///GO:0003735,GO:0002181///GO:0006412///GO:0006412,GO:0005737///GO:0022625///GO:0005622///GO:0030...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18857708,852500,852500,"""MI:0463""(biogrid)",2932.0,DER1,-0.120,0.584,0.790,0.468,0.003,Der1p,DER1,YBR201W,"Chromosome II, NC_001134.8 (623576..624211)",misfolded protein binding,ER-associated ubiquitin-dependent protein cata...,Hrd1p ubiquitin ligase ERAD-L complex///endopl...,GO:0051787,GO:0030433///GO:0030433///GO:0070843///GO:0015...,GO:0000839///GO:0005783///GO:0005789///GO:0005...
19709657,1466537,1466537,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None
18824936,853207,853207,"""MI:0463""(biogrid)",7665.0,MAL11,-1.056,0.501,0.879,0.067,-0.692,alpha-glucoside permease,MAL11,YGR289C,"Chromosome VII, NC_001139.9 (1073963..1075813,...",alpha-glucoside:proton symporter activity///gl...,carbohydrate transport///disaccharide cataboli...,integral component of membrane///integral comp...,GO:0005352///GO:0005355///GO:0005364///GO:0022...,GO:0008643///GO:0046352///GO:0046323///GO:0035...,GO:0016021///GO:0016021///GO:0005887///GO:0016020
18824930,852429,852429,"""MI:0463""(biogrid)",5231.0,AGP2,-1.762,-0.223,1.414,0.733,-0.785,Agp2p,AGP2,YBR132C,"Chromosome II, NC_001134.8 (499652..501442, co...",L-amino acid transmembrane transporter activit...,amino acid transmembrane transport///amino aci...,endoplasmic reticulum///endoplasmic reticulum ...,GO:0015179///GO:0015171///GO:0015297///GO:0003...,GO:0003333///GO:0003333///GO:0006865///GO:1902...,GO:0005783///GO:0005789///GO:0000329///GO:

In [31]:
for i in range(0, 20):
  column_vals = p4c.get_table_columns(table='node', columns=['shared name', 'name', 'psi-mi', 'ID_REF', 'IDENTIFIER', 'GSM1029', 'GSM1030', 'GSM1032', 'GSM1033', 'GSM1034', 'Gene title', 'Gene symbol', 'Platform_ORF', 'Chromosome annotation', 'GO:Function', 'GO:Process', 'GO:Component', 'GO:Function ID', 'GO:Process ID', 'GO:Component ID'])
  print(f'Get columns {i}: {my_uuid} {"successful" if all_columns.equals(column_vals) else "failed"}')



Get columns 0: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 1: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 2: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 3: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 4: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 5: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 6: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 7: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 8: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 9: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 10: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 11: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 12: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 13: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 14: d3ef5dc4-a593-4bf3-b639-6c1e951191d8 successful
Get columns 15: d3ef5dc4-a593-4bf3-b639-6c1e951191